# YouTube Reviews EDA and Modeling

Getting Youtube headphone review video CC texts, and creating a dataset from this of earbuds to build a basic recommendation system.

In [1]:
import numpy as np
import pandas as pd
import json
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

from YouTubeReviewScraper import YouTubeReviewData

## Collecting Review Data V1

This is an updated version to get the headphone reviews. Here we can get a review for a headphone alongside the title of the video, youtube link, channel name, and the video id.

In [ ]:
""" This code will create a json file with your api key which you will need to load it in, in the next cell
# Define your API key
api_key = "your_api_key_here"

# Create a dictionary with the API key
data = {"api_key": api_key}

# Specify the file path
file_path = "config.json"

# Write the dictionary to the JSON file
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=2)

print(f"JSON file '{file_path}' created with API key.")
"""
pass

In [2]:
file_path = "config.json"

# Read the content of the JSON file
with open(file_path, 'r') as json_file:
    config_data = json.load(json_file)

# Extract the API key from the config file, which should also be in gitignore
api_key = config_data.get('api_key', None)

#will call this many time for each search query to get the reviews
scraper = YouTubeReviewData(api_key)
max_results = 7

In [3]:
sony_search_query = 'sony xm4 earbuds review'
headphone_name = ' '.join([word for word in sony_search_query.split() if word != 'review'])

In [7]:
af = pd.DataFrame(scraper.search_videos(sony_search_query, 2))

In [9]:
af['headphone'] = headphone_name
af

,video_id,title,video_link,channel_name,review_text,headphone
0,0L4tO4HDjnU,Sony WF-1000XM4 Earbuds Review - 6 Months Later,https://www.youtube.com/watch?v=0L4tO4HDjnU,6 Months Later,"The Sony WF-1000XM4 earbuds, which is a mouth...",sony xm4 earbuds
1,hIWn_RjN-Wg,I&#39;m ditching my AirPods Pros - Sony WF-100...,https://www.youtube.com/watch?v=hIWn_RjN-Wg,ShortCircuit,(wind rushing) (slow music) - As much as I lo...,sony xm4 earbuds


In [10]:
af2 = pd.DataFrame(scraper.search_videos('Galaxy Buds2 Pro review', 2))

In [11]:
af2['headphone'] = ' '.join([word for word in 'Galaxy Buds2 Pro review'.split() if word != 'review'])

In [12]:
af2

,video_id,title,video_link,channel_name,review_text,headphone
0,N8LH414Ut-4,Samsung Galaxy Buds 2 Pro Review - 6 Months Later,https://www.youtube.com/watch?v=N8LH414Ut-4,6 Months Later,It's been six months since I bought a pair o...,Galaxy Buds2 Pro
1,vluEm6XJWjs,NEW GALAXY BUDS 2 PRO - Samsung&#39;s disrupti...,https://www.youtube.com/watch?v=vluEm6XJWjs,Mike O'Brien,all right the galaxy buds 2 pro wow these ear...,Galaxy Buds2 Pro


In [18]:
testing_concat = pd.concat([af, af2], axis=0)
testing_concat.reset_index(drop=True)

,video_id,title,video_link,channel_name,review_text,headphone
0,0L4tO4HDjnU,Sony WF-1000XM4 Earbuds Review - 6 Months Later,https://www.youtube.com/watch?v=0L4tO4HDjnU,6 Months Later,"The Sony WF-1000XM4 earbuds, which is a mouth...",sony xm4 earbuds
1,hIWn_RjN-Wg,I&#39;m ditching my AirPods Pros - Sony WF-100...,https://www.youtube.com/watch?v=hIWn_RjN-Wg,ShortCircuit,(wind rushing) (slow music) - As much as I lo...,sony xm4 earbuds
2,N8LH414Ut-4,Samsung Galaxy Buds 2 Pro Review - 6 Months Later,https://www.youtube.com/watch?v=N8LH414Ut-4,6 Months Later,It's been six months since I bought a pair o...,Galaxy Buds2 Pro
3,vluEm6XJWjs,NEW GALAXY BUDS 2 PRO - Samsung&#39;s disrupti...,https://www.youtube.com/watch?v=vluEm6XJWjs,Mike O'Brien,all right the galaxy buds 2 pro wow these ear...,Galaxy Buds2 Pro


In [25]:
search_queries = ['sony xm4 earbuds review', 'Galaxy Buds2 Pro review', 'Sennheiser MTW3 review',
                 'Bose Quietcomfort Earbuds 2 review', 'Bose Quietcomfort Earbuds review', 'Beats Fit Pro Earbuds review',
                 'AirPods Pro 2 Earbuds review', 'Sony Linkbuds S review', 'Sony Linkbuds original review', 
                  'Pixel Buds Pro review', 'Soundcore Liberty 3 review', 'AirPods 3 review', 
                  'Jabra Elite 7 Pro review', 'Sony WF-1000XM5 review', '1MORE Evo review', 'lg tone tf8 review']

In [21]:
empty_df = pd.DataFrame()

pd.concat([empty_df, af2], axis=0)

,video_id,title,video_link,channel_name,review_text,headphone
0,N8LH414Ut-4,Samsung Galaxy Buds 2 Pro Review - 6 Months Later,https://www.youtube.com/watch?v=N8LH414Ut-4,6 Months Later,It's been six months since I bought a pair o...,Galaxy Buds2 Pro
1,vluEm6XJWjs,NEW GALAXY BUDS 2 PRO - Samsung&#39;s disrupti...,https://www.youtube.com/watch?v=vluEm6XJWjs,Mike O'Brien,all right the galaxy buds 2 pro wow these ear...,Galaxy Buds2 Pro


In [26]:
#initialize an empty dataframe to add each dataframe per search query as we iterate
youtube_review_df = pd.DataFrame()

for query in search_queries:
    query_df = pd.DataFrame(scraper.search_videos(query, 7))
    # get the headphone name from the query by removing the word 'review'
    headphone_name = ' '.join([word for word in query.split() if word != 'review'])
    query_df['headphone'] = headphone_name
    
    # add the dataframe to the empty initialized one as we iterate
    youtube_review_df = pd.concat([youtube_review_df, query_df], axis=0)

An error occurred: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=vSBsswDEj7g! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (vSBsswDEj7g) transcripts are available in the following languages:

(MANUALLY CREATED)
 - nl ("Dutch")[TRANSLATABLE]
 - en-US ("English (United States)")[TRANSLATABLE]
 - fr ("French")[TRANSLATABLE]
 - de ("German")[TRANSLATABLE]
 - it ("Italian")[TRANSLATABLE]
 - pl ("Polish")[TRANSLATABLE]
 - pt ("Portuguese")[TRANSLATABLE]
 - es ("Spanish")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simpli

In [30]:
youtube_review_df = youtube_review_df.reset_index(drop=True)
youtube_review_df.head(10)

,video_id,title,video_link,channel_name,review_text,headphone
0,0L4tO4HDjnU,Sony WF-1000XM4 Earbuds Review - 6 Months Later,https://www.youtube.com/watch?v=0L4tO4HDjnU,6 Months Later,"The Sony WF-1000XM4 earbuds, which is a mouth...",sony xm4 earbuds
1,hIWn_RjN-Wg,I&#39;m ditching my AirPods Pros - Sony WF-100...,https://www.youtube.com/watch?v=hIWn_RjN-Wg,ShortCircuit,(wind rushing) (slow music) - As much as I lo...,sony xm4 earbuds
2,fMw1h4hKT_E,Sony WF-1000XM4 review: silence to my ears,https://www.youtube.com/watch?v=fMw1h4hKT_E,The Verge,so it's been almost two years since sony rele...,sony xm4 earbuds
3,mpOv9h0L9Cc,The Most Advanced Earbuds? Sony WF-1000XM4 Rev...,https://www.youtube.com/watch?v=mpOv9h0L9Cc,UrAvgConsumer,[Music] what's going on guys it's your averag...,sony xm4 earbuds
4,Rknl4uIOyfY,Sony WF-1000XM4 true wireless earbuds full review,https://www.youtube.com/watch?v=Rknl4uIOyfY,GSMArena Official,hey what's up guys will here for gsm arena we...,sony xm4 earbuds
5,Hxz_r0vAPoo,Worth buying Sony WF-1000XM4 in 2023? 🤔,https://www.youtube.com/watch?v=Hxz_r0vAPoo,Aaron X Loud and Wireless,[Music] hey guys so is the Sony wfos xm4 stil...,sony xm4 earbuds
6,QC7IRar8Ytw,Almost Ideal Premium TWS | Sony WF 1000XM4 Review,https://www.youtube.com/watch?v=QC7IRar8Ytw,Geekyranjit,[Music] hi guys this is ranjit and in this vi...,sony xm4 earbuds
7,N8LH414Ut-4,Samsung Galaxy Buds 2 Pro Review - 6 Months Later,https://www.youtube.com/watch?v=N8LH414Ut-4,6 Months Later,It's been six months since I bought a pair o...,Galaxy Buds2 Pro
8,vluEm6XJWjs,NEW GALAXY BUDS 2 PRO - Samsung&#39;s disrupti...,https://www.youtube.com/watch?v=vluEm6XJWjs,Mike O'Brien,all right the galaxy buds 2 pro wow these ear...,Galaxy Buds2 Pro
9,Xs3-9Pz4ED4,Samsung Galaxy Buds 2 Pro Review | Better than...,https://www.youtube.com/watch?v=Xs3-9Pz4ED4,Tech Spurt,hello gorgeous peeps I'm Chris from Tex burn ...,Galaxy Buds2 Pro


In [32]:
youtube_review_df.to_csv('youtube_reviews_v2.csv', index=False)

Just testing the functions from the youtube scraper file here

In [24]:
from googleapiclient.discovery import build
def search_videos(search_query, max_results=5):
    youtube = build('youtube', 'v3', developerKey=api_key)
    search_response = youtube.search().list(
        q=search_query,
        type='video',
        part='id, snippet',
        maxResults=max_results
    ).execute()

    """
    video_ids = [result['id']['videoId'] for result in search_response.get('items', [])]
    titles = [result['snippet']['title'] for result in search_response.get('items', [])]

    #any videos with vs in the title means it is not a direct review of the specific headphone, so we will not want those
    strings_to_check = ["VS", "vs", "Vs"]
    indices_to_remove = [i for i, title in enumerate(titles) if any(s in title for s in strings_to_check)]
    # Remove items from the video_ids list using the indices
    video_ids = [video_ids[i] for i in range(len(video_ids)) if i not in indices_to_remove]
    """
    videos_info = []
    for result in search_response.get('items', []):
        video_id = result['id']['videoId']
        title = result['snippet']['title']
        video_link = f'https://www.youtube.com/watch?v={video_id}'
        channel_name = result['snippet']['channelTitle']

        # Check and remove unwanted titles
        strings_to_check = ["VS", "vs", "Vs"]
        if not any(s in title for s in strings_to_check):
            videos_info.append({
                'video_id': video_id,
                'title': title,
                'video_link': video_link,
                'channel_name': channel_name
            })     


    return videos_info

def review_lists(search_query, max_results=7):
    # Get the video ids for the top x reviews
    video_info = search_videos(search_query=search_query, max_results=max_results)
    video_ids = [dct['video_id'] for dct in video_info]

    #video_review_texts = [fetch_captions(video_ids[i]) for i in range(0, len(video_ids))]

    return video_info

In [25]:
search_videos(sony_search_query)

[{'video_id': '0L4tO4HDjnU',
  'title': 'Sony WF-1000XM4 Earbuds Review - 6 Months Later',
  'video_link': 'https://www.youtube.com/watch?v=0L4tO4HDjnU',
  'channel_name': '6 Months Later'},
 {'video_id': 'hIWn_RjN-Wg',
  'title': 'I&#39;m ditching my AirPods Pros - Sony WF-1000XM4 Wireless Earbuds',
  'video_link': 'https://www.youtube.com/watch?v=hIWn_RjN-Wg',
  'channel_name': 'ShortCircuit'},
 {'video_id': 'fMw1h4hKT_E',
  'title': 'Sony WF-1000XM4 review: silence to my ears',
  'video_link': 'https://www.youtube.com/watch?v=fMw1h4hKT_E',
  'channel_name': 'The Verge'},
 {'video_id': 'mpOv9h0L9Cc',
  'title': 'The Most Advanced Earbuds? Sony WF-1000XM4 Review!',
  'video_link': 'https://www.youtube.com/watch?v=mpOv9h0L9Cc',
  'channel_name': 'UrAvgConsumer'},
 {'video_id': 'Rknl4uIOyfY',
  'title': 'Sony WF-1000XM4 true wireless earbuds full review',
  'video_link': 'https://www.youtube.com/watch?v=Rknl4uIOyfY',
  'channel_name': 'GSMArena Official'}]

## Collecting Review Data V2

Below is the first version of collecting the review texts - it will likely be broken now. V1 is more complete in that the YouTubeReviewScraper file now gets the review text on its own and some other data about each youube review.

In [16]:
def get_headphone_reviews(search_query, headphone_name, max_results):
    review_texts = scraper.review_lists(search_query, max_results)
    
    # Create a DataFrame
    data = {
        "Headphone_Name": [headphone_name] * len(review_texts),
        "Sony_Review_Text": review_texts
    }

    reviews_df = pd.DataFrame(data)
    
    return reviews_df

In [18]:
df = get_headphone_reviews(sony_search_query, headphone_name, max_results)
df.head()

NameError: name 'result' is not defined

In [131]:
dataframes = []

for query in search_queries:
    #split the query into a list of words, then make a new list without the word review, and turn it into a list again
    #this will give a string with just the name and without the word review in it
    headphone_name = ' '.join([word for word in query.split() if word != 'review'])
    
    df = get_headphone_reviews(query, headphone_name, max_results)
    
    dataframes.append(df)
    
    final_dataframe = pd.concat(dataframes, ignore_index=True)

An error occurred: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=vSBsswDEj7g! This is most likely caused by:

No transcripts were found for any of the requested language codes: ('en',)

For this video (vSBsswDEj7g) transcripts are available in the following languages:

(MANUALLY CREATED)
 - nl ("Dutch")[TRANSLATABLE]
 - en-US ("English (United States)")[TRANSLATABLE]
 - fr ("French")[TRANSLATABLE]
 - de ("German")[TRANSLATABLE]
 - it ("Italian")[TRANSLATABLE]
 - pl ("Polish")[TRANSLATABLE]
 - pt ("Portuguese")[TRANSLATABLE]
 - es ("Spanish")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simpli

In [135]:
final_dataframe

,Headphone_Name,Sony_Review_Text
0,sony xm4 earbuds,"The Sony WF-1000XM4 earbuds, which is a mouth..."
1,sony xm4 earbuds,(wind rushing)\n(slow music) - As much as I l...
2,sony xm4 earbuds,[Music] what's going on guys it's your averag...
3,sony xm4 earbuds,so it's been almost two years since sony rele...
4,sony xm4 earbuds,[Music] hey guys so is the Sony wfos xm4 stil...
...,...,...
107,Buy LG TONE TF8,these are algae's first sports earbuds they a...
108,Buy LG TONE TF8,all right guys so this video is sponsored by ...
109,Buy LG TONE TF8,the market of true wireless earbuds has grown...
110,Buy LG TONE TF8,last year i reviewed lg's tone free fp 8 and ...


In [136]:
final_dataframe.to_csv('youtube_reviews.csv', index=False)

## Text Cleaning

In [23]:
YT_df = pd.read_csv('youtube_reviews.csv')

YT_df.rename(columns={'Sony_Review_Text': 'Review_Text'}, inplace=True)

YT_df = YT_df.dropna()

YT_df.head(3)

,Headphone_Name,Review_Text
0,sony xm4 earbuds,"The Sony WF-1000XM4 earbuds, which is a mouth..."
1,sony xm4 earbuds,(wind rushing)\n(slow music) - As much as I l...
2,sony xm4 earbuds,[Music] what's going on guys it's your averag...


In [11]:
stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer()

emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                           u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                           u"\U0001F700-\U0001F77F"  # Alchemical Symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing"
                           u"\U0001FC00-\U0001FCFF"  # St. George's Flag
                           u"\U0001F004-\U0001F0CF"  # CJK Compatibility Ideographs
                           u"\U0001F170-\U0001F251"  # Enclosed Ideographic Supplement
                           "]+", flags=re.UNICODE)

In [12]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    #remove emoji's
    text = emoji_pattern.sub(r'', text)
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and apply stemming
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    # Join the words back into a string
    preprocessed_text = ' '.join(words)
    return preprocessed_text

In [24]:
YT_df['Review_Text'] = YT_df['Review_Text'].apply(lambda x: preprocess_text(x))

In [26]:
YT_df.head(3)

,Headphone_Name,Review_Text
0,sony xm4 earbuds,soni wfxm earbud mouth name sit toward top ear...
1,sony xm4 earbuds,wind rush slow music much love airpod pro admi...
2,sony xm4 earbuds,music what go guy averag consum today final go...


## Building Recommendation System

In [2]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import gensim.downloader as api

from transformers import BertTokenizer, BertModel
import torch

Getting glove embeddings to vectorize the text

In [130]:
#load glove vectors and save them to avoid loading time
glove_model = api.load("glove-wiki-gigaword-300")
model_file = "glove_model.gensim"
glove_model.save(model_file)

In [3]:
from gensim.models import KeyedVectors
model_file = "glove_model.gensim"
#load pre-saved glove vectors to avoid loading time
glove_model = KeyedVectors.load(model_file)

In [132]:
def get_document_embedding(text, glove_model):
    # Tokenize the text and calculate the average word embedding
    tokens = text.lower().split()
    word_embeddings = [glove_model[word] for word in tokens if word in glove_model]
    if not word_embeddings:
        # If no valid words are found in the model, return a zero vector
        return np.zeros(glove_model.vector_size)
    return np.mean(word_embeddings, axis=0)

In [133]:
YT_df['ReviewEmbedding'] = YT_df['Review_Text'].apply(lambda x: get_document_embedding(x, glove_model))

YT_df.head(3)

,Headphone_Name,Review_Text,ReviewEmbedding
0,sony xm4 earbuds,soni wfxm earbud mouth name sit toward top ear...,"[-0.09994754, 0.03841405, -0.13169473, -0.0995..."
1,sony xm4 earbuds,wind rush slow music much love airpod pro admi...,"[-0.07816244, 0.014967105, -0.13430771, -0.103..."
2,sony xm4 earbuds,music what go guy averag consum today final go...,"[-0.06597834, 0.025261255, -0.110347696, -0.04..."


Grouping by headphone model and averaging the reviews so that we will get recommendations based on an average of the models. Not sure if this is the best method, but it is what we will do for now.

In [53]:
product_reviews_grouped = YT_df.groupby('Headphone_Name')['ReviewEmbedding'].apply(list).reset_index()
product_reviews_grouped['ProductEmbedding'] = product_reviews_grouped['ReviewEmbedding'].apply(lambda x: np.mean(x, axis=0))

In [54]:
product_reviews_grouped.head(3)

,Headphone_Name,ReviewEmbedding,ProductEmbedding
0,1MORE Evo,"[[-0.07878432, 0.015500491, -0.10244502, -0.03...","[-0.06718464, 0.051035076, -0.12470848, -0.064..."
1,AirPods 3,"[[-0.10321395, 0.061115436, -0.09049448, -0.06...","[-0.08022302, 0.038006034, -0.0901948, -0.0595..."
2,AirPods Pro 2 Earbuds,"[[-0.093394734, 0.08188138, -0.116774656, -0.0...","[-0.070139505, 0.06712871, -0.089367665, -0.06..."


In [56]:
#don't need the ReviewEmbedding column, it is just the list of the original embeddings - but we only need the averaged one now
averaged_embeddings = product_reviews_grouped.drop(columns = 'ReviewEmbedding', axis = 1)

#can't save multidimensional objects to csv so have to convert arrays to lists
averaged_embeddings['ProductEmbedding'] = averaged_embeddings['ProductEmbedding'].apply(lambda arr: arr.tolist())

averaged_embeddings.to_csv('averaged_embeddings.csv', index=False)

In [6]:
from ast import literal_eval

#converters will handle the list column and make sure it isn't a string
averaged_embeddings_df = pd.read_csv('averaged_embeddings.csv', converters={'ProductEmbedding': literal_eval})

In [129]:
user_input = 'I want a headphone with the very best noise cancellation and also incredibly comforatable. Good battery life too'
user_input_embedding = get_document_embedding(user_input, glove_model)

### Cosine Similarity

In [71]:
product_reviews_grouped['CosineSimilarity'] = product_reviews_grouped['ProductEmbedding'].apply(
    lambda x: cosine_similarity([user_input_embedding], [x])[0][0]
)

# Get product recommendations based on similarity
top_recommendations = product_reviews_grouped.nlargest(5, 'CosineSimilarity')

print("Recommended products based on user input:")
print(top_recommendations[['Headphone_Name', 'CosineSimilarity']])

Recommended products based on user input:
                 Headphone_Name  CosineSimilarity
14          Soundcore Liberty 3          0.909348
4     Bose Quietcomfort Earbuds          0.908522
15             sony xm4 earbuds          0.908121
10              Sennheiser MTW3          0.908089
5   Bose Quietcomfort Earbuds 2          0.907921


In [113]:
cos_sim = averaged_embeddings_df['ProductEmbedding'].apply(
    lambda x: cosine_similarity([user_input_embedding], [x])[0][0]
)

cos_sim = cos_sim.rename("CosineSimilarity")

pd.concat([averaged_embeddings_df, cos_sim], axis = 1)

,Headphone_Name,ProductEmbedding,CosineSimilarity
0,1MORE Evo,"[-0.06718464195728302, 0.051035076379776, -0.1...",0.906417
1,AirPods 3,"[-0.08022301644086838, 0.03800603374838829, -0...",0.907703
2,AirPods Pro 2 Earbuds,"[-0.07013950496912003, 0.06712871044874191, -0...",0.906755
3,Beats Fit Pro Earbuds,"[-0.06112515553832054, 0.056722816079854965, -...",0.898826
4,Bose Quietcomfort Earbuds,"[-0.06966901570558548, 0.05094931274652481, -0...",0.908522
5,Bose Quietcomfort Earbuds 2,"[-0.06322579085826874, 0.05420923978090286, -0...",0.907921
6,Buy LG TONE TF8,"[-0.07814421504735947, 0.05790264585188457, -0...",0.905804
7,Galaxy Buds2 Pro,"[-0.07899948209524155, 0.08394227176904678, -0...",0.905966
8,Jabra Elite 7 Pro,"[-0.0671735405921936, 0.0659547969698906, -0.1...",0.902456
9,Pixel Buds Pro,"[-0.07331924140453339, 0.09483595192432404, -0...",0.903866


In [134]:
def get_recommendation(user_input, reviews_df, glove_model):
    user_input_embedding = get_document_embedding(user_input, glove_model)
    
    #get the cosine similarity
    cos_sim = reviews_df['ProductEmbedding'].apply(lambda x: cosine_similarity([user_input_embedding], [x])[0][0])
    cos_sim = cos_sim.rename("CosineSimilarity")
    
    #concatenate the dataframes so we can get the headphone names
    rec_df = pd.concat([reviews_df, cos_sim], axis = 1)
    
    # Get product recommendations based on similarity
    top_recommendations = rec_df.nlargest(5, 'CosineSimilarity')
    
    return top_recommendations.drop(columns = ['ProductEmbedding'], axis = 1)

In [13]:
get_recommendation(user_input, averaged_embeddings_df, glove_model)

NameError: name 'get_recommendation' is not defined

In [4]:
#testing the class for getting the recommendations
from Get_Recommendations import HeadphoneRecommendations

In [7]:
# Initialize the recommendations class
recommendations = HeadphoneRecommendations(model_file=model_file, reviews_df=averaged_embeddings_df)

# Get recommendations for a user input
user_input = "I want the best expensive black headphones with good sound quality and top of the line noise cancellation features. Also good battery."
top_recommendations = recommendations.get_recommendation(user_input)

#resetting index to show top 5 and only the headphone names and not the cosine similarites
print(top_recommendations)

0      Bose Quietcomfort Earbuds
1               sony xm4 earbuds
2          AirPods Pro 2 Earbuds
3    Bose Quietcomfort Earbuds 2
4                Sennheiser MTW3
Name: Headphone_Name, dtype: object


In [ ]:
#run this in the command line to run the script
#!python Get_Recommendations.py --model glove_model.gensim --reviews averaged_embeddings.csv

## Scratch Code for BERT

In [51]:


def get_bert_word_embeddings(text, model_name, max_seq_length=512):
    # Load the BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Tokenize and truncate the text
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt')

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract embeddings from the BERT model
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    return embeddings


# Example usage
text = "This is an example sentence for BERT embeddings."
model_name = "bert-base-uncased"  # Replace with the desired BERT model name
bert_embeddings = get_bert_word_embeddings(text, model_name)


NameError: name 'model' is not defined

In [50]:
def apply_bert_embedding(text):
    return get_bert_word_embeddings(text, model_name = "bert-base-uncased")

YT_df['ReviewBERTEmbedding'] = YT_df['Review_Text'].apply(apply_bert_embedding)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (1411 > 512). Running this sequence through the model wil

RuntimeError: The expanded size of the tensor (1411) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 1411].  Tensor sizes: [1, 512]

## Scratch Code for Testing

In [84]:
max_results = 6

In [46]:
api_key = api_key
youtube = build('youtube', 'v3', developerKey=api_key)

In [94]:
search_response = youtube.search().list(
    q="Galaxy Buds2 Pro review",
    type='video',
    part='id, snippet',
    maxResults=max_results
).execute()
search_response.get('items', [])[0]
video_ids = [result['id']['videoId'] for result in search_response.get('items', [])]
titles = [result['snippet']['title'] for result in search_response.get('items', [])]

In [90]:
titles

['Samsung Galaxy Buds 2 Pro Review - 6 Months Later',
 'NEW GALAXY BUDS 2 PRO - Samsung&#39;s disruptive earbuds',
 'Samsung Galaxy Buds 2 Pro Review: My Ears Are Happy Now',
 'Samsung Galaxy Buds 2 Pro Review | Better than Pixel Buds Pro?',
 'Samsung Galaxy Buds2 Pro Review in 2023: what changed?',
 'Samsung Galaxy Buds 2 Pro review | Great till they ain&#39;t']

In [91]:
strings_to_check = ["Later", "Happy", "changed"]

In [96]:
# Get the indices of titles that contain specified strings
indices_to_remove = [i for i, title in enumerate(titles) if any(s in title for s in strings_to_check)]

# Remove items from the video_ids list using the indices
video_ids = [video_ids[i] for i in range(len(video_ids)) if i not in indices_to_remove]

In [95]:
video_ids

['N8LH414Ut-4',
 'vluEm6XJWjs',
 'dg4pGN0-h7Y',
 'Xs3-9Pz4ED4',
 'vSBsswDEj7g',
 'JQgwxgti6m8']

In [97]:
video_ids

['vluEm6XJWjs', 'Xs3-9Pz4ED4', 'JQgwxgti6m8']

In [66]:
search_response.get('items', [])[0]['snippet']['title']

'Sony WF-1000XM4 Earbuds Review - 6 Months Later'